# Imports

In [ ]:
import requests
import getpass
import pickle
import io
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

# Login
https://www.statistikdaten.bayern.de/genesis/online?Menu=Anmeldung#abreadcrumb

In [ ]:
username = input()

In [ ]:
password = getpass.getpass()

## Test login

In [ ]:
class GenesisApi:

    def __init__(self, username, password, polling_rate=5):
        self.username = username
        self.password = password
        self.polling_rate = polling_rate

        self.__base_url = 'https://www.statistikdaten.bayern.de/genesisWS/rest/2020/'

        self.__base_params = {
            'username': username,
            'password': password,
            'language': 'de'
        }

        self.__default_table_params = self.__base_params.copy()
        self.__default_table_params.update({
            'name': '',
            'area': 'all',
            'compress': 'false',
            'transpose': 'false',
            'startyear': '',
            'endyear': '',
            'timeslices': '',
            'regionalvariable': '',
            'regionalkey': '',
            'classifyingkey1': '',
            'classifyingvariable2': '',
            'classifyingkey2': '',
            'classifyingvariable3': '',
            'classifyingkey3': '',
            'job': 'true'
        })

        self.__default_jobs_params = self.__base_params.copy()
        self.__default_jobs_params.update({
            'selection': '',
            'searchcriterion': 'code',
            'sortcriterion': 'code',
            'type': 'all',
            'area': 'all',
            'pagelength': '100'
        })

        self.__default_result_params = self.__base_params.copy()
        self.__default_result_params.update({
            'name': '',
            'area': 'all',
            'compress': 'false'
        })

    def check_login(self):
        response = requests.get(self.__base_url + 'helloworld/logincheck', params=self.__base_params)
        b'{"Status":"Sie wurden erfolgreich an- und abgemeldet!","Username":"GB3U65P838"}'
        try:
            return response.json()['Status'] == 'Sie wurden erfolgreich an- und abgemeldet!'
        except Exception as e:
            return False

    def get_table(self, name, startyear=''):
        startyear = str(startyear)

        params = self.__default_table_params.copy()
        params['name'] = name
        params['startyear'] = startyear

        response = requests.get(self.__base_url + 'data/table', params=params)

        data = response.json()
        code = data['Status']['Code']
        if (code == 0):  # Success
            return data
        elif (code == 99):  # Table is too big a job has been created
            print('Table is too big, created a job.')
            result_name = data['Status']['Content'].split(':', 1)[1][1:]
            return self.get_job_result(result_name)
        else:
            params['password'] = '***'
            print('Error requesting ' + name + ' with params:', params, 'response:', data)
            return data

    def is_job_ready(self, name):
        params = self.__default_jobs_params.copy()
        params['selection'] = 'Werteabruf ' + name

        response = requests.get(self.__base_url + 'catalogue/jobs', params=params)
        try:
            return response.json()['List'][0]['State'] == 'Fertig'
        except Exception as e:
            return False

    def delete_job_result(self, name):
        params = self.__default_result_params.copy()
        params['name'] = name
        response = requests.get(self.__base_url + 'profile/removeResult', params=params)
        return response

    def get_job_result(self, name):
        params = self.__default_result_params.copy()
        params['name'] = name

        while (not self.is_job_ready(name)):
            print('Data is not ready waiting ' + str(self.polling_rate) + ' seconds longer.')
            time.sleep(self.polling_rate)

        response = requests.get(self.__base_url + 'data/result', params=params)
        self.delete_job_result(name)
        return response.json()


In [ ]:
genesis = GenesisApi(username, password)
genesis.check_login()

# Download data

Note: This takes a long time

In [ ]:
responses_demographic = {}

for year in range(1980, 2020 + 1):
    print('Requesting table for the year ' + str(year))
    response = genesis.get_table('12411-003r', year)
    print('Got data')
    responses_demographic[str(year)] = response

In [ ]:
responses_area = {}

# 33111-201r 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2009, 2010, 2011, 2012, 2013
# 33111-001r 2014 - 2020

for year in [1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2009, 2010, 2011, 2012, 2013]:
    print('Requesting table for the year ' + str(year))
    response = genesis.get_table('33111-201r', year)
    print('Got data')
    responses_area[str(year)] = response

for year in range(2014, 2020 + 1):
    print('Requesting table for the year ' + str(year))
    response = genesis.get_table('33111-001r', year)
    print('Got data')
    responses_area[str(year)] = response

# Convert to DataFrame

In [ ]:
def convert_to_dataframe(response, start_at_line, date_line, header_line):
    raw_content = response['Object']['Content']
    content = raw_content.split('\n', start_at_line)
    date = content[date_line].split(';', 1)[0]
    csv = io.StringIO(content[header_line] + '\n' + content[start_at_line].split('\n__________', 1)[0])
    df = pd.read_csv(csv, ';')
    df['date'] = pd.to_datetime(date, format='%d.%m.%Y')
    return df

## Demographic

In [ ]:
dfs = list()
for year, response in responses_demographic.items():
    df = convert_to_dataframe(response, start_at_line=6, date_line=4, header_line=5)
    dfs.append(df)

df_demographic = pd.concat(dfs, axis=0, ignore_index=True)

column_names = df_demographic.columns.values
column_names[0] = 'AGS'
column_names[1] = 'Gemeinde'
df_demographic.columns = column_names

df_demographic['Gemeinde'] = df_demographic['Gemeinde'].str.strip()
df_demographic['Insgesamt'] = pd.to_numeric(df_demographic['Insgesamt'], errors='coerce')
df_demographic['männlich'] = pd.to_numeric(df_demographic['männlich'], errors='coerce')
df_demographic['weiblich'] = pd.to_numeric(df_demographic['weiblich'], errors='coerce')

## Area

In [ ]:
dfs = list()
for year, response in responses_area.items():
    df = convert_to_dataframe(response, start_at_line=10, date_line=5, header_line=8)

    column_names = df.columns.values
    column_names[0] = 'AGS'
    column_names[1] = 'Gemeinde'
    df.columns = column_names

    for column_name in column_names[2: len(column_names) - 1]:
        df[column_name] = pd.to_numeric(df[column_name].str.replace(',', '.'), errors='coerce')

    df['Gemeinde'] = df['Gemeinde'].str.strip()

    dfs.append(df)

df_area = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
df_area

# Save and load data

In [ ]:
df_demographic.to_pickle('df_demographic.pickle')
df_area.to_pickle('df_area.pickle')

with open('responses_demographic.pickle', 'wb') as f:
    pickle.dump(responses_demographic, f, pickle.HIGHEST_PROTOCOL)

with open('responses_area.pickle', 'wb') as f:
    pickle.dump(responses_area, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
df_demographic = pd.read_pickle('df_demographic.pickle')
df_area = pd.read_pickle('df_area.pickle')

with open('responses_demographic.pickle', 'rb') as f:
    responses_demographic = pickle.load(f)

with open('responses_area.pickle', 'rb') as f:
    responses_area = pickle.load(f)

## Categorize

In [ ]:
used = [
    "Wohnen",
    "Grünanlage",
    "Friedhof",
    "sonstige Erholungsfläche",
    "11000 Wohnbaufläche",
    "18100 Sportanlage",
    "18200 Freizeitanlage",
    "19000 Friedhof",
    "18300 Erholungsfläche",
    "18400 Grünanlage",

    "Gewerbe, Industrie",
    "Betriebsfläche (ohne Abbauland)",
    "Flächen anderer Nutzung (ohne Unland, Friedhof)",
    "12100 Industrie und Gewerbe",
    "12200 Handel und Dienstleistung",
    "12300 Versorgungsanlage",
    "12400 Entsorgung",
    "13000 Halde",
    "17000 Fläche besonderer funktionaler Prägung",

    "16000 Fläche gemischter Nutzung",
    "sonstige Gebäude- und Freifläche",

    "Straße, Weg, Platz",
    "sonstige Verkehrsfläche",
    "21000 Straßenverkehr",
    "22000 Weg",
    "23000 Platz",
    "24000 Bahnverkehr",
    "25000 Flugverkehr",
    "26000 Schiffsverkehr",
]
unused = [
    "Abbauland",
    "14000 Bergbaubetrieb",
    "15000 Tagebau, Grube, Steinbruch",
    "Moor",
    "Landwirtschaftsfläche (ohne Moor, Heide)",
    "Heide",
    "Waldfläche",
    "Unland",
    "31100 Ackerland",
    "31200 Grünland",
    "31300 Gartenland",
    "31400 Weingarten",
    "31500 Obstplantage",
    "32000 Wald",
    "33000 Gehölz",
    "34000 Heide",
    "35000 Moor",
    "36000 Sumpf",
    "37000 Unland, Vegetationslose Fläche",

    "Wasserfläche",
    "42000 Hafenbecken",
    "41000 Fließgewässer",
    "43000 Stehendes Gewässer",
]

In [ ]:
categories = {
    "living": [  # Wohnen
        "Wohnen",
        "11000 Wohnbaufläche",
    ],

    "industry": [  # Industrie / Gewerbe
        "Gewerbe, Industrie",
        "Betriebsfläche (ohne Abbauland)",
        "Flächen anderer Nutzung (ohne Unland, Friedhof)",
        "12100 Industrie und Gewerbe",
        "12200 Handel und Dienstleistung",
        "12300 Versorgungsanlage",
        "12400 Entsorgung",
        "13000 Halde",
        "17000 Fläche besonderer funktionaler Prägung",
    ],

    "misc_industry_living": [  # Mischflächen
        "Grünanlage",
        "Friedhof",
        "sonstige Erholungsfläche",
        "sonstige Gebäude- und Freifläche",
        "16000 Fläche gemischter Nutzung",
        "18100 Sportanlage",
        "18200 Freizeitanlage",
        "18300 Erholungsfläche",
        "18400 Grünanlage",
        "19000 Friedhof",
    ],

    "transport_infrastructure": [
        "Straße, Weg, Platz",
        "sonstige Verkehrsfläche",
        "21000 Straßenverkehr",
        "22000 Weg",
        "23000 Platz",
        "24000 Bahnverkehr",
        "25000 Flugverkehr",
        "26000 Schiffsverkehr",
    ],

    "nature": [
        "Moor",
        "Landwirtschaftsfläche (ohne Moor, Heide)",
        "Heide",
        "Waldfläche",
        "Unland",
        "31100 Ackerland",
        "31200 Grünland",
        "31300 Gartenland",
        "31400 Weingarten",
        "31500 Obstplantage",
        "32000 Wald",
        "33000 Gehölz",
        "34000 Heide",
        "35000 Moor",
        "36000 Sumpf",
        "37000 Unland, Vegetationslose Fläche",
    ],

    "water": [
        "Wasserfläche",
        "42000 Hafenbecken",
        "41000 Fließgewässer",
        "43000 Stehendes Gewässer",
    ],

    "mining": [
        "Abbauland",
        "14000 Bergbaubetrieb",
        "15000 Tagebau, Grube, Steinbruch",
    ],
}

In [ ]:
# Check if we classified all columns and used each only once
all_columns = set(df_area.columns)

for l in categories.values():
    all_columns = all_columns - set(l)

all_columns = all_columns - {'AGS', 'Gemeinde', 'Insgesamt', 'date'}

if len(all_columns) != 0:
    print("The categories", all_columns, "have not yet been categorized.")

if len(set(df_area.columns) - set(used) - set(unused) - {'AGS', 'Gemeinde', 'Insgesamt', 'date'}) != 0:
    print("The categories", all_columns, "have not yet been categorized.")

for ((name1, l1), (name2, l2)) in itertools.combinations(categories.items(), 2):
    if not set(l1).isdisjoint(l2):
        print(name1, "and", name2, "contain the same category.")

if not set(used).isdisjoint(unused):
    print("used and unused contain the same category")

In [ ]:
categories['used_area'] = used
#categories['unused_area'] = unused

In [ ]:
for (name, category) in categories.items():
    df_area[name] = df_area.loc[:, category].sum(axis=1)
    df_area[name + '_percent'] = df_area[name] / df_area['Insgesamt']

In [ ]:
to_delete = set()

for (name, category) in categories.items():
    to_delete.update(category)

df_area.drop(to_delete, axis=1, inplace=True)

## Rename columns

In [ ]:
df_area.rename(columns={"Insgesamt": "total", "Gemeinde": "municipality"}, inplace=True)

## Filter unused municipalities

In [ ]:
df_area = df_area[df_area["AGS"] <= 9999]

## Merge demographic data

In [ ]:
df_demographic.drop(["männlich", "weiblich"], axis=1, inplace=True)
df_demographic.rename(columns={"Gemeinde": "municipality", "Insgesamt": "demographic"}, inplace=True)

In [ ]:
df_area = pd.merge(df_area, df_demographic, how='left', on=['AGS', 'municipality', 'date'])

## Export to JSON

In [ ]:
df_export = df_area.copy()
df_export['date'] = df_export['date'].dt.strftime('%d.%m.%Y')

with open("../src/data/data.json", "w", encoding="utf-8") as f:
    df_export.to_json(f, orient="records", force_ascii=False)

In [ ]:
def group_by_and_export(data, filename):
    json = "{"
    first = True
    for date, inner_json in data.groupby("date").apply(lambda x: x.to_json(orient='records')).items():
        if not first:
            json = json + ","
        else:
            first = False
        json = json + '"' + date + '":' + inner_json
    json = json + "}"

    with open("../src/data/" + filename + ".json", "w", encoding="utf-8") as f:
        f.write(json)
        f.flush()

In [ ]:
group_by_and_export(df_export[df_export["AGS"] <= 99], "RBYear")
group_by_and_export(df_export[df_export["AGS"] > 99], "LKYear")

In [ ]:
# Generate code for https://sankeymatic.com/build/
# Then manually create 2 svg graphs
# One with white text color and onw with black text color
# Label position should be "Right side"
# Width is 1200px and height is 900 px
# Margins are all 12, except right is 290
# Space between Nodes is 10 px with a width of 8
# Flows should use the source node's color

df_sankey = df_area[(df_area["AGS"] == 9) & (df_area["date"] == pd.to_datetime("31.12.2020", format='%d.%m.%Y'))]

sankey = ""
sankey += "Gesamtfläche [29] Verbrauchte Fläche\n"
sankey += "Gesamtfläche [23] Unverbrauchte Fläche\n"

category_names = {
    "living": "Wohnfläche",
    "industry": "Industrie/Gewerbe",
    "misc_industry_living": "Sonstiges",
    "transport_infrastructure": "Verkehrsflächen",
    "nature": "Natur",
    "water": "Wasser",
    "mining": "Bergbau"
}

category_colors = {
    "living": "#FF375F",
    "industry": "#FF9F0A",
    "misc_industry_living": "#FFD60A",
    "transport_infrastructure": "#8E8E93",
    "nature": "#30D158",
    "water": "#0A84FF",
    "mining": "#BF5AF2"
}

cat2 = categories.copy()
del cat2['used_area']

for category_name, name in category_names.items():
    size = str(len(cat2[category_name]))
    if category_name == "water" or category_name == "nature" or category_name == "mining":
        sankey += "Unverbrauchte Fläche [" + size + "] " + name + "\n"
    else:
        sankey += "Verbrauchte Fläche [" + size + "] " + name + "\n"
    sankey += ":" + name + " " + category_colors[category_name] + "\n"

for category_name, name in category_names.items():
    sub_categories = cat2[category_name]
    color = category_colors[category_name]
    for category in sub_categories:
        sankey += name + " [1] " + category + "\n"
        sankey += ":" + category + " " + color + "\n"

sankey += ":Verbrauchte Fläche #F00\n"
sankey += ":Unverbrauchte Fläche #0F0\n"
sankey += ":Gesamtfläche #004477\n"

with open("sankey", "w", encoding="utf-8") as f:
    f.write(sankey)
    f.flush()

# Basic graphs

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
#ax.set(yscale="log")
g = sns.lineplot(data=df_area[df_area['municipality'] == 'Augsburg (Krfr.St)'], style='municipality', x='date',
                 y='demographic', ax=ax)
g.set_title('Einwohner')
g.set(ylim=(0, None))
#sns.lineplot(data=df_demographic, style='Gemeinde', x='date', y='Insgesamt', ax=ax)#, ylim=(0,300000))

In [ ]:
_, ax = plt.subplots(figsize=(7, 7))
#ax.set(yscale="log")
g = sns.lineplot(data=df_area[df_area['municipality'] == 'Augsburg (Krfr.St)'], style='municipality', x='date',
                 y='nature_percent', ax=ax)
g.set_title('Grünflächen')
g.set(ylim=(0, 1))

In [ ]:
_, ax = plt.subplots(figsize=(7, 7))
#ax.set(yscale="log")
g = sns.lineplot(
    data=df_area[(df_area['municipality'] == 'Augsburg (Krfr.St)') | (df_area['municipality'] == 'Bayern')],
    style='municipality', x='date',
    y='used_area_percent', ax=ax)
g.set_title('Verbrauchte Fläche')
g.set(ylim=(0, 1))

In [ ]:
df_tmp = df_area.copy()
df_tmp = df_tmp[((df_tmp["AGS"] <= 99) & (df_tmp["AGS"] >= 10)) | (df_tmp["municipality"] == 'Augsburg (Krfr.St)') | (
        df_tmp["municipality"] == 'München, Landeshauptstadt')]

f, ax = plt.subplots(figsize=(14, 14))
#ax.set(yscale="log")
sns.set_theme(style="whitegrid")
g = sns.lineplot(
    data=df_tmp,
    palette='bright',
    hue='municipality', x='date',
    y='used_area_percent', ax=ax)
g.set_title('Verbrauchte Fläche')
g.set(ylim=(0, 1))

f.savefig('flächen.jpg')

In [ ]:
df_tmp = df_area.copy()
df_tmp = df_tmp[((df_tmp["AGS"] <= 99) & (df_tmp["AGS"] >= 10)) | (df_tmp["municipality"] == 'Augsburg (Krfr.St)') | (
        df_tmp["municipality"] == 'München, Landeshauptstadt')]
#ax.set(yscale="log")
sns.set_theme(style="whitegrid")
g = sns.relplot(
    data=df_tmp,
    palette='bright',
    col='municipality', x='date',
    kind="line",
    col_wrap=3,
    y='used_area_percent', height=6)
#g.set_title('Verbrauchte Fläche')
g.set(ylim=(0, 1))

plt.savefig('flächen.jpg')



In [ ]:
df_area['municipality']

In [ ]:
df_area

In [ ]:
gem = ['Bayern']  #, 'Oberbayern', 'Schwaben']
size = 10
f, axs = plt.subplots(len(gem), 1, figsize=(size * 3, len(gem) * size * 3))

#df_area_2 = df_area_2[df_area_2['date'] > pd.to_datetime("1.1.2010", format='%d.%m.%Y')]

for i in range(0, len(gem)):
    g = df_area[(df_area['municipality'] == gem[i])].plot.area(
        x='date',
        y=['living_percent', 'industry_percent', 'transport_infrastructure_percent', 'nature_and_water_percent',
           'miscellaneous_percent'],
        stacked=True,
        ax=(axs if len(gem) == 1 else axs[i]))
    g.set_title('Flächen in ' + gem[i])
    g.set(ylim=(0, None))

plt.savefig('flächen.jpg')